In [6]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"


### Connecting to mlflow client

In [8]:

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

### Listing all experiments on client

In [10]:
#client.list_experiments()
# list experiments has been removed, use search_experiments
client.search_experiments()

[<Experiment: artifact_location=('/Users/andreanicolas/Library/CloudStorage/GoogleDrive-andreanicolas91@gmail.com/My '
  'Drive/ASU_ComputerScience/MLOps_studies/02-experiment-tracking/mlruns/1'), creation_time=1716340686867, experiment_id='1', last_update_time=1716340686867, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716339311631, experiment_id='0', last_update_time=1716339311631, lifecycle_stage='active', name='Default', tags={}>]

### Creating a new experiment

In [11]:
# to create a new experiment:
client.create_experiment(name="my-cool-experiment")

'2'

### Obtaining top 5 best runs

In [18]:
from mlflow.entities import ViewType
# get best runs
runs = client.search_runs(
    experiment_ids= 1, #  id 1 is for nyc-taxi-experiment
    filter_string="metrics.rmse < 5.2", #default is ""
    run_view_type=ViewType.ACTIVE_ONLY, #default
    max_results=10, # top 10
    order_by=["metrics.rmse ASC"] # RMSE ascending
)

In [20]:
# print the best 5 runs
for run in runs:
    print(f"run name: {run.info.run_name}, run id: {run.info.run_id},rmse: {run.data.metrics['rmse']:.4f}")

run name: fun-bird-728, run id: 3a1b47272275407cb31762aa65763ffe,rmse: 5.1121
run name: legendary-sloth-321, run id: 42b2669b03d9485fb14b40a2ebbc1653,rmse: 5.1157
run name: dapper-finch-532, run id: 4e68fd0dcf3941ec88dbfa4f62f7cc69,rmse: 5.1187
run name: serious-fox-828, run id: bfce02dcde384b48bdc8c59926b4b36e,rmse: 5.1190
run name: vaunted-crab-10, run id: 3fd3beb2a281479294f7fc646149c2c1,rmse: 5.1195
run name: glamorous-turtle-248, run id: 0b983bbfdc7148a3951d7bce9c997594,rmse: 5.1196
run name: amazing-yak-769, run id: 1a1936ba97f04dc488782a8cc6242c7b,rmse: 5.1196
run name: salty-rook-257, run id: c77157dfb8314558a44e404e402c1930,rmse: 5.1196
run name: clumsy-lynx-332, run id: 75727bda700b42c3961acea9e6d1f336,rmse: 5.1244
run name: rogue-colt-880, run id: 480546989b644d959ce918447c230389,rmse: 5.1251


### Registering the model

In [23]:
import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

# for glamorous turtle:
run_id = "0b983bbfdc7148a3951d7bce9c997594"
model_uri = f"runs:/{run_id}/models_mlflow"
mlflow.register_model(model_uri=model_uri,name='nyc-taxi-regressor')

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1716412936661, current_stage='None', description=None, last_updated_timestamp=1716412936661, name='nyc-taxi-regressor', run_id='0b983bbfdc7148a3951d7bce9c997594', run_link=None, source=('/Users/andreanicolas/Library/CloudStorage/GoogleDrive-andreanicolas91@gmail.com/My '
 'Drive/ASU_ComputerScience/MLOps_studies/02-experiment-tracking/mlruns/1/0b983bbfdc7148a3951d7bce9c997594/artifacts/models_mlflow'), status='READY', status_message=None, tags={}, user_id=None, version=3>

#### Transitioning from one stage to another

In [28]:
model_name = 'nyc-taxi-regressor'
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")


version: 2, stage: Staging
version: 3, stage: None


/var/folders/fm/8gkkcxbn0y3896pxrr1p0nqh0000gn/T/ipykernel_3802/3344490539.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [44]:
model_version = 3
model_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=model_stage,
    archive_existing_versions=False
)

/var/folders/fm/8gkkcxbn0y3896pxrr1p0nqh0000gn/T/ipykernel_3802/1093745685.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716412936661, current_stage='Production', description='The model version 3 was transitioned to Staging on 2024-05-22', last_updated_timestamp=1716417436300, name='nyc-taxi-regressor', run_id='0b983bbfdc7148a3951d7bce9c997594', run_link=None, source=('/Users/andreanicolas/Library/CloudStorage/GoogleDrive-andreanicolas91@gmail.com/My '
 'Drive/ASU_ComputerScience/MLOps_studies/02-experiment-tracking/mlruns/1/0b983bbfdc7148a3951d7bce9c997594/artifacts/models_mlflow'), status='READY', status_message=None, tags={}, user_id=None, version=3>

### Updating model info

In [45]:
from datetime import datetime
date = datetime.today().date()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {model_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1716412936661, current_stage='Production', description='The model version 3 was transitioned to Production on 2024-05-22', last_updated_timestamp=1716417455901, name='nyc-taxi-regressor', run_id='0b983bbfdc7148a3951d7bce9c997594', run_link=None, source=('/Users/andreanicolas/Library/CloudStorage/GoogleDrive-andreanicolas91@gmail.com/My '
 'Drive/ASU_ComputerScience/MLOps_studies/02-experiment-tracking/mlruns/1/0b983bbfdc7148a3951d7bce9c997594/artifacts/models_mlflow'), status='READY', status_message=None, tags={}, user_id=None, version=3>

## Promote Stating into Prod and moving existing Prod to archive

### Create functions to do preprocessing and testing
- January - training
- February - validation
- march - testing

In [39]:
from sklearn.metrics import root_mean_squared_error
import pandas as pd

# function to read and filter data
def read_dataframe(filename):
    df = pd.read_parquet(filename) #read data
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime # calculate duration
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60) # convert to mins
    mask = ((df.duration >=1) & (df.duration<=60)) # select only significant data
    df = df[mask]
    categorical = ['PULocationID', 'DOLocationID'] # pickup and dropoff location
    df[categorical] = df[categorical].astype(str) # convert to string/object to prevent labeling

    return df

# function to preprocess by combining two datasets into one
def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

# function to test the existing model via RMSE
def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

#### Read in test dataframe and download preprocessor from mlflow's glamorous turtle 

In [40]:
df= read_dataframe("../data/green_tripdata_2023-03.parquet")

In [41]:
client.download_artifacts(run_id=run_id, path='preprocessor',dst_path='.')

'/Users/andreanicolas/Library/CloudStorage/GoogleDrive-andreanicolas91@gmail.com/My Drive/ASU_ComputerScience/MLOps_studies/02-experiment-tracking/preprocessor'

In [42]:
import pickle 
with open("preprocessor/preprocessor.b","rb") as f_in:
    dv = pickle.load(f_in)

#### Preprocess the dataset using already fitted/transformed preprocessor

In [43]:
X_test = preprocess(df,dv)

#### Test the model

In [46]:
target = "duration"
y_test = df[target].values

In [47]:
%time test_model(name=model_name, stage="Production",X_test=X_test,y_test=y_test)

/Users/andreanicolas/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
/Users/andreanicolas/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [15:38:30] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: user 7.44 s, sys: 226 ms, total: 7.67 s
Wall time: 1.66 s


{'rmse': 5.3531981028447575}

In [48]:
%time test_model(name=model_name, stage="Staging",X_test=X_test,y_test=y_test)

/Users/andreanicolas/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
/Users/andreanicolas/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [15:41:12] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: user 7.26 s, sys: 71.6 ms, total: 7.33 s
Wall time: 1.17 s


{'rmse': 5.3531981028447575}

#### Upgrading and archiving

In [51]:
from datetime import datetime
date = datetime.today().date()

model_name = 'nyc-taxi-regressor'
model_version = 1
model_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=model_stage,
    archive_existing_versions=True
)

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {model_stage} on {date}"
)

/var/folders/fm/8gkkcxbn0y3896pxrr1p0nqh0000gn/T/ipykernel_3802/3641299958.py:7: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716407098344, current_stage='Production', description='The model version 1 was transitioned to Production on 2024-05-22', last_updated_timestamp=1716417939984, name='nyc-taxi-regressor', run_id='0b983bbfdc7148a3951d7bce9c997594', run_link='', source=('/Users/andreanicolas/Library/CloudStorage/GoogleDrive-andreanicolas91@gmail.com/My '
 'Drive/ASU_ComputerScience/MLOps_studies/02-experiment-tracking/mlruns/1/0b983bbfdc7148a3951d7bce9c997594/artifacts/models_mlflow'), status='READY', status_message=None, tags={'model': 'fastest'}, user_id=None, version=1>